# Based
- https://www.kaggle.com/code/honganzhu/cmi-piu-competition?scriptVersionId=201912528 Version44 LB0.492

 If you find this notebook useful, please upvote this and the based one.

In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from pytorch_tabnet.tab_model import TabNetRegressor
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_str_lengths(10000)

polars.config.Config

In [2]:
#from IPython.display import display as ipydisplay, HTML
#ipydisplay(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 80em; }</style>"))

In [3]:
SEED = 42
n_splits = 5

# Feature Engineering

- **Feature Selection**: The dataset contains features related to physical characteristics (e.g., BMI, Height, Weight), behavioral aspects (e.g., internet usage), and fitness data (e.g., endurance time). 
- **Categorical Feature Encoding**: Categorical features are mapped to numerical values using custom mappings for each unique category within the dataset. This ensures compatibility with machine learning algorithms that require numerical input.
- **Time Series Aggregation**: Time series statistics (e.g., mean, standard deviation) from the actigraphy data are computed and merged into the main dataset to create additional features for model training.


In [4]:
path = "I:/Kaggle/child-mind-institute-problematic-internet-use/"

In [5]:
train = pd.read_csv(path + 'train.csv', dtype={'id': str})
test = pd.read_csv(path + 'test.csv', dtype={'id': str})
sample = pd.read_csv(path + 'sample_submission.csv')

In [6]:
print(train.shape)
train.head()

(3960, 82)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [7]:
print(test.shape)
test.head()

(20, 59)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [8]:
sample

id  sii
0   00008ff9    0
1   000fd460    1
2   00105258    2
3   00115b9f    3
4   0016bb22    0
5   001f3379    1
6   0038ba98    2
7   0068a485    3
8   0069fbed    0
9   0083e397    1
10  0087dd65    2
11  00abe655    3
12  00ae59c9    0
13  00af6387    1
14  00bd4359    2
15  00c0cd71    3
16  00d56d4b    0
17  00d9913d    1
18  00e6167c    2
19  00ebc35d    3

In [9]:
data_dict = pd.read_csv(path + 'data_dictionary.csv')
print(data_dict.shape)
data_dict

(81, 6)


Instrument  \
0                                      Identifier   
1                                    Demographics   
2                                    Demographics   
3                                    Demographics   
4              Children's Global Assessment Scale   
5              Children's Global Assessment Scale   
6                               Physical Measures   
7                               Physical Measures   
8                               Physical Measures   
9                               Physical Measures   
10                              Physical Measures   
11                              Physical Measures   
12                              Physical Measures   
13                              Physical Measures   
14               FitnessGram Vitals and Treadmill   
15               FitnessGram Vitals and Treadmill   
16               FitnessGram Vitals and Treadmill   
17               FitnessGram Vitals and Treadmill   
18                              FitnessGram Child   
19                              FitnessGram Child   
20                              FitnessGram Child   
21                              FitnessGram Child   
22                              FitnessGram Child   
23                              FitnessGram Child   
24                              FitnessGram Child   
25                              FitnessGram Child   
26                              FitnessGram Child   
27                              FitnessGram Child   
28                              FitnessGram Child   
29                              FitnessGram Child   
30                              FitnessGram Child   
31                              FitnessGram Child   
32                              FitnessGram Child   
33                Bio-electric Impedance Analysis   
34                Bio-electric Impedance Analysis   
35                Bio-electric Impedance Analysis   
36                Bio-electric Impedance Analysis   
37                Bio-electric Impedance Analysis   
38                Bio-electric Impedance Analysis   
39                Bio-electric Impedance Analysis   
40                Bio-electric Impedance Analysis   
41                Bio-electric Impedance Analysis   
42                Bio-electric Impedance Analysis   
43                Bio-electric Impedance Analysis   
44                Bio-electric Impedance Analysis   
45                Bio-electric Impedance Analysis   
46                Bio-electric Impedance Analysis   
47                Bio-electric Impedance Analysis   
48                Bio-electric Impedance Analysis   
49                Bio-electric Impedance Analysis   
50  Physical Activity Questionnaire (Adolescents)   
51  Physical Activity Questionnaire (Adolescents)   
52     Physical Activity Questionnaire (Children)   
53     Physical Activity Questionnaire (Children)   
54           Parent-Child Internet Addiction Test   
55           Parent-Child Internet Addiction Test   
56           Parent-Child Internet Addiction Test   
57           Parent-Child Internet Addiction Test   
58           Parent-Child Internet Addiction Test   
59           Parent-Child Internet Addiction Test   
60           Parent-Child Internet Addiction Test   
61           Parent-Child Internet Addiction Test   
62           Parent-Child Internet Addiction Test   
63           Parent-Child Internet Addiction Test   
64           Parent-Child Internet Addiction Test   
65           Parent-Child Internet Addiction Test   
66           Parent-Child Internet Addiction Test   
67           Parent-Child Internet Addiction Test   
68           Parent-Child Internet Addiction Test   
69           Parent-Child Internet Addiction Test   
70           Parent-Child Internet Addiction Test   
71           Parent-Child Internet Addiction Test   
72           Parent-Child Internet Addiction Test   
73           Parent-Child Internet Addiction Test   
74           Parent-Child Internet Addiction Test   
75         

In [10]:
train['id'].nunique()

3960

In [11]:
test['id'].nunique()

20

In [12]:
train_target = train[test.columns]

In [13]:
print(train_target.shape)
train_target.head()

(3960, 59)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [14]:
concat_df = pd.concat([train_target, test], ignore_index=True)
print(concat_df.shape)
concat_df.head()

(3980, 59)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [15]:
concat_df['total_num_nan'] = concat_df.isna().sum(axis=1)

In [16]:
concat_df['total_num_nan'].describe()

count    3980.000000
mean       26.142462
std        17.257902
min         3.000000
25%         9.000000
50%        24.000000
75%        41.000000
max        55.000000
Name: total_num_nan, dtype: float64

In [17]:
nan_df = concat_df[concat_df['total_num_nan'] == concat_df['total_num_nan'].max()]
print(nan_df.shape)
nan_df.head()

(199, 60)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
42   023e1b6a                    Summer               13                0   
67   040d103f                    Spring                8                1   
84   04ccb838                    Summer                7                0   
107  05dc60ef                    Spring               12                1   
148  0882b065                    Summer               11                0   

    CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  \
42          NaN              NaN             NaN           NaN   
67          NaN              NaN             NaN           NaN   
84          NaN              NaN             NaN           NaN   
107         NaN              NaN             NaN           NaN   
148         NaN              NaN             NaN           NaN   

     Physical-Height  Physical-Weight  Physical-Waist_Circumference  \
42               NaN              NaN                           NaN   
67               NaN              NaN                           NaN   
84               NaN              NaN                           NaN   
107              NaN              NaN                           NaN   
148              NaN              NaN                           NaN   

     Physical-Diastolic_BP  Physical-HeartRate  Physical-Systolic_BP  \
42                     NaN                 NaN                   NaN   
67                     NaN                 NaN                   NaN   
84                     NaN                 NaN                   NaN   
107                    NaN                 NaN                   NaN   
148                    NaN                 NaN                   NaN   

    Fitness_Endurance-Season  Fitness_Endurance-Max_Stage  \
42                       NaN                          NaN   
67                       NaN                          NaN   
84                       NaN                          NaN   
107                      NaN                          NaN   
148                      NaN                          NaN   

     Fitness_Endurance-Time_Mins  Fitness_Endurance-Time_Sec FGC-Season  \
42                           NaN                         NaN        NaN   
67                           NaN                         NaN        NaN   
84                           NaN                         NaN        NaN   
107                          NaN                         NaN        NaN   
148                          NaN                         NaN        NaN   

     FGC-FGC_CU  FGC-FGC_CU_Zone  FGC-FGC_GSND  FGC-FGC_GSND_Zone  \
42          NaN              NaN           NaN                NaN   
67          NaN              NaN           NaN                NaN   
84          NaN              NaN           NaN                NaN   
107         NaN              NaN           NaN                NaN   
148         NaN              NaN           NaN                NaN   

     FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  FGC-FGC_PU_Zone  FGC-FGC_SRL  \
42           NaN               NaN         NaN              NaN          NaN   
67           NaN               NaN         NaN              NaN          NaN   
84           NaN               NaN         NaN              NaN          NaN   
107          NaN               NaN         NaN              NaN          NaN   
148          NaN               NaN         NaN              NaN          NaN   

     FGC-FGC_SRL_Zone  FGC-FGC_SRR  FGC-FGC_SRR_Zone  FGC-FGC_TL  \
42                NaN          NaN               NaN         NaN   
67                NaN          NaN               NaN         NaN   
84                NaN          NaN               NaN         NaN   
107               NaN          NaN               NaN         NaN   
148               NaN          NaN               NaN         NaN   

     FGC-FGC_TL_Zone BIA-Season  BIA-BIA_Activity_Level_num  BIA-BIA_BMC  \
42               NaN        NaN                         NaN          NaN   
67              

In [18]:
for col in concat_df.columns:
    if col == 'id':
        continue
    new_col_name = col + '_isnan'
    concat_df[new_col_name] = concat_df[col].isna().astype(int)

In [19]:
print(concat_df.shape)
concat_df.head()

(3980, 119)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [20]:
instruments = data_dict['Instrument'].unique().tolist()
instruments.remove('Identifier')
print(len(instruments))
instruments

11


['Demographics',
 "Children's Global Assessment Scale",
 'Physical Measures',
 'FitnessGram Vitals and Treadmill',
 'FitnessGram Child',
 'Bio-electric Impedance Analysis',
 'Physical Activity Questionnaire (Adolescents)',
 'Physical Activity Questionnaire (Children)',
 'Parent-Child Internet Addiction Test',
 'Sleep Disturbance Scale',
 'Internet Use']

In [21]:
for instrument in instruments:
    if 'Parent-Child Internet Addiction Test' == instrument:
        continue
    instrument_df = data_dict[data_dict['Instrument'] == instrument]
    field_list = instrument_df['Field'].unique().tolist()
    concat_df[instrument + '_num_nan'] = concat_df[field_list].isna().sum(axis=1)    

In [22]:
print(concat_df.shape)
concat_df.head()

(3980, 129)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [23]:
train_parquet_id_folders_list = os.listdir(path + 'series_train.parquet/')
print(len(train_parquet_id_folders_list))
train_parquet_id_folders_list

996


['id=00115b9f',
 'id=001f3379',
 'id=00f332d1',
 'id=01085eb3',
 'id=012cadd8',
 'id=012e3869',
 'id=029a19c9',
 'id=02cebf33',
 'id=02cf7384',
 'id=035c96dd',
 'id=03a9019b',
 'id=0417c91e',
 'id=045a0a94',
 'id=04afb6f9',
 'id=04bb1a76',
 'id=04cb2c30',
 'id=04d06a9c',
 'id=04f094a8',
 'id=051680a0',
 'id=055156e2',
 'id=059eed01',
 'id=05bbed1b',
 'id=05db1b9b',
 'id=05e94f88',
 'id=063b16fc',
 'id=064e8da5',
 'id=0668373f',
 'id=067b9287',
 'id=06c8b2fb',
 'id=06eb8adb',
 'id=070386b2',
 'id=07048067',
 'id=0720320e',
 'id=0745c390',
 'id=080698b7',
 'id=082df556',
 'id=08577722',
 'id=08660d12',
 'id=089a6b19',
 'id=09331bb8',
 'id=0983654b',
 'id=099e3614',
 'id=09b29113',
 'id=0a418b57',
 'id=0a431608',
 'id=0b4014f0',
 'id=0b50f3fa',
 'id=0b518637',
 'id=0b7d7aec',
 'id=0b7d9da6',
 'id=0bd89878',
 'id=0bfe4fc9',
 'id=0c4e2d68',
 'id=0c4eabac',
 'id=0c98f822',
 'id=0cbe4e54',
 'id=0d01bbf2',
 'id=0d279d77',
 'id=0d4800c8',
 'id=0d4fa8f4',
 'id=0d570d3e',
 'id=0d924604',
 'id=0e3

In [24]:
train_parquet_id_list = [v.replace('id=', '') for v in train_parquet_id_folders_list]
print(len(train_parquet_id_list))
train_parquet_id_list

996


['00115b9f',
 '001f3379',
 '00f332d1',
 '01085eb3',
 '012cadd8',
 '012e3869',
 '029a19c9',
 '02cebf33',
 '02cf7384',
 '035c96dd',
 '03a9019b',
 '0417c91e',
 '045a0a94',
 '04afb6f9',
 '04bb1a76',
 '04cb2c30',
 '04d06a9c',
 '04f094a8',
 '051680a0',
 '055156e2',
 '059eed01',
 '05bbed1b',
 '05db1b9b',
 '05e94f88',
 '063b16fc',
 '064e8da5',
 '0668373f',
 '067b9287',
 '06c8b2fb',
 '06eb8adb',
 '070386b2',
 '07048067',
 '0720320e',
 '0745c390',
 '080698b7',
 '082df556',
 '08577722',
 '08660d12',
 '089a6b19',
 '09331bb8',
 '0983654b',
 '099e3614',
 '09b29113',
 '0a418b57',
 '0a431608',
 '0b4014f0',
 '0b50f3fa',
 '0b518637',
 '0b7d7aec',
 '0b7d9da6',
 '0bd89878',
 '0bfe4fc9',
 '0c4e2d68',
 '0c4eabac',
 '0c98f822',
 '0cbe4e54',
 '0d01bbf2',
 '0d279d77',
 '0d4800c8',
 '0d4fa8f4',
 '0d570d3e',
 '0d924604',
 '0e3b0bd2',
 '0ec403df',
 '0ed2a3c8',
 '0eddd8e5',
 '0f6b71c4',
 '0fbb3ca5',
 '101774f0',
 '101c27f1',
 '108b8659',
 '10ad8c48',
 '10e46254',
 '113f5798',
 '11739de2',
 '119d7562',
 '119e181e',

In [25]:
test_parquet_id_folders_list = os.listdir(path + 'series_test.parquet/')
print(len(test_parquet_id_folders_list))
test_parquet_id_folders_list

2


['id=00115b9f', 'id=001f3379']

In [26]:
test_parquet_id_list = [v.replace('id=', '') for v in test_parquet_id_folders_list]
print(len(test_parquet_id_list))
test_parquet_id_list

2


['00115b9f', '001f3379']

In [27]:
train_temp_df = concat_df.head(train.shape[0])
print(train_temp_df.shape)
train_temp_df.head()

(3960, 129)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [28]:
test_temp_df = concat_df.tail(test.shape[0])
print(test_temp_df.shape)
test_temp_df.head()

(20, 129)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
3960  00008ff9                      Fall                5                0   
3961  000fd460                    Summer                9                0   
3962  00105258                    Summer               10                1   
3963  00115b9f                    Winter                9                0   
3964  0016bb22                    Spring               18                1   

     CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  \
3960      Winter             51.0            Fall     16.877316   
3961         NaN              NaN            Fall     14.035590   
3962        Fall             71.0            Fall     16.648696   
3963        Fall             71.0          Summer     18.292347   
3964      Summer              NaN             NaN           NaN   

      Physical-Height  Physical-Weight  Physical-Waist_Circumference  \
3960             46.0             50.8                           NaN   
3961             48.0             46.0                          22.0   
3962             56.5             75.6                           NaN   
3963             56.0             81.6                           NaN   
3964              NaN              NaN                           NaN   

      Physical-Diastolic_BP  Physical-HeartRate  Physical-Systolic_BP  \
3960                    NaN                 NaN                   NaN   
3961                   75.0                70.0                 122.0   
3962                   65.0                94.0                 117.0   
3963                   60.0                97.0                 117.0   
3964                    NaN                 NaN                   NaN   

     Fitness_Endurance-Season  Fitness_Endurance-Max_Stage  \
3960                      NaN                          NaN   
3961                      NaN                          NaN   
3962                     Fall                          5.0   
3963                   Summer                          6.0   
3964                      NaN                          NaN   

      Fitness_Endurance-Time_Mins  Fitness_Endurance-Time_Sec FGC-Season  \
3960                          NaN                         NaN       Fall   
3961                          NaN                         NaN       Fall   
3962                          7.0                        33.0       Fall   
3963                          9.0                        37.0     Summer   
3964                          NaN                         NaN        NaN   

      FGC-FGC_CU  FGC-FGC_CU_Zone  FGC-FGC_GSND  FGC-FGC_GSND_Zone  \
3960         0.0              0.0           NaN                NaN   
3961         3.0              0.0           NaN                NaN   
3962        20.0              1.0          10.2                1.0   
3963        18.0              1.0           NaN                NaN   
3964         NaN              NaN           NaN                NaN   

      FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  FGC-FGC_PU_Zone  FGC-FGC_SRL  \
3960          NaN               NaN         0.0              0.0          7.0   
3961          NaN               NaN         5.0              0.0         11.0   
3962         14.7               2.0         7.0              1.0         10.0   
3963          NaN               NaN         5.0              0.0          7.0   
3964          NaN               NaN         NaN              NaN          NaN   

      FGC-FGC_SRL_Zone  FGC-FGC_SRR  FGC-FGC_SRR_Zone  FGC-FGC_TL  \
3960               0.0          6.0               0.0         6.0   
3961               1.0         11.0               1.0         3.0   
3962               1.0         10.0               1.0         5.0   
3963               0.0          7.0               0.0         7.0   
3964               NaN          NaN               NaN         NaN   

      FGC-FGC_TL_Zone BIA-Season  BIA-BIA_Activity_Level_num  BIA-BIA_BMC  \
3960              1.0       Fall      

In [29]:
train_temp_df['has_parquet'] = train_temp_df['id'].isin(train_parquet_id_list).astype(int)
print(train_temp_df.shape)
train_temp_df.head()

(3960, 130)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [30]:
test_temp_df['has_parquet'] = test_temp_df['id'].isin(test_parquet_id_list).astype(int)
print(test_temp_df.shape)
test_temp_df.head()

(20, 130)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
3960  00008ff9                      Fall                5                0   
3961  000fd460                    Summer                9                0   
3962  00105258                    Summer               10                1   
3963  00115b9f                    Winter                9                0   
3964  0016bb22                    Spring               18                1   

     CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  \
3960      Winter             51.0            Fall     16.877316   
3961         NaN              NaN            Fall     14.035590   
3962        Fall             71.0            Fall     16.648696   
3963        Fall             71.0          Summer     18.292347   
3964      Summer              NaN             NaN           NaN   

      Physical-Height  Physical-Weight  Physical-Waist_Circumference  \
3960             46.0             50.8                           NaN   
3961             48.0             46.0                          22.0   
3962             56.5             75.6                           NaN   
3963             56.0             81.6                           NaN   
3964              NaN              NaN                           NaN   

      Physical-Diastolic_BP  Physical-HeartRate  Physical-Systolic_BP  \
3960                    NaN                 NaN                   NaN   
3961                   75.0                70.0                 122.0   
3962                   65.0                94.0                 117.0   
3963                   60.0                97.0                 117.0   
3964                    NaN                 NaN                   NaN   

     Fitness_Endurance-Season  Fitness_Endurance-Max_Stage  \
3960                      NaN                          NaN   
3961                      NaN                          NaN   
3962                     Fall                          5.0   
3963                   Summer                          6.0   
3964                      NaN                          NaN   

      Fitness_Endurance-Time_Mins  Fitness_Endurance-Time_Sec FGC-Season  \
3960                          NaN                         NaN       Fall   
3961                          NaN                         NaN       Fall   
3962                          7.0                        33.0       Fall   
3963                          9.0                        37.0     Summer   
3964                          NaN                         NaN        NaN   

      FGC-FGC_CU  FGC-FGC_CU_Zone  FGC-FGC_GSND  FGC-FGC_GSND_Zone  \
3960         0.0              0.0           NaN                NaN   
3961         3.0              0.0           NaN                NaN   
3962        20.0              1.0          10.2                1.0   
3963        18.0              1.0           NaN                NaN   
3964         NaN              NaN           NaN                NaN   

      FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  FGC-FGC_PU_Zone  FGC-FGC_SRL  \
3960          NaN               NaN         0.0              0.0          7.0   
3961          NaN               NaN         5.0              0.0         11.0   
3962         14.7               2.0         7.0              1.0         10.0   
3963          NaN               NaN         5.0              0.0          7.0   
3964          NaN               NaN         NaN              NaN          NaN   

      FGC-FGC_SRL_Zone  FGC-FGC_SRR  FGC-FGC_SRR_Zone  FGC-FGC_TL  \
3960               0.0          6.0               0.0         6.0   
3961               1.0         11.0               1.0         3.0   
3962               1.0         10.0               1.0         5.0   
3963               0.0          7.0               0.0         7.0   
3964               NaN          NaN               NaN         NaN   

      FGC-FGC_TL_Zone BIA-Season  BIA-BIA_Activity_Level_num  BIA-BIA_BMC  \
3960              1.0       Fall      

In [31]:
# https://www.kaggle.com/code/antoninadolgorukova/cmi-piu-actigraphy-data-eda
entropy = lambda x: -(x / x.sum() * np.log(x / x.sum() + 1e-9)).sum()

In [32]:
def feat_engi(df, target_cols):
    stats_df = df[target_cols].describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99]).T
    count = stats_df['count'][0]
    stats_df.drop('count', axis=1, inplace=True)
    
    column_names_list = []
    for idx in stats_df.index:
        for col in stats_df.columns:
            column_names_list.append(str(idx) + '_' + str(col))

    reshaped_df = pd.DataFrame(stats_df.values.reshape(-1)).T.reset_index(drop=True)
    reshaped_df.columns = column_names_list

    reshaped_df['count'] = count

    return reshaped_df

In [33]:
def groupby_feat_engi(df, col_to_group, target_col):
    stats_df = pd.DataFrame(
        df.groupby(col_to_group)[target_col].describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99])
    )

    stats_df['sum'] = df.groupby(col_to_group)[target_col].sum()
    stats_df['entropy'] = df.groupby(col_to_group)[target_col].apply(entropy)
    
    column_names_list = []
    for idx in stats_df.index:
        for col in stats_df.columns:
            column_names_list.append(target_col + '_' + col_to_group + '_' + str(idx) + '_' + str(col))

    reshaped_df = pd.DataFrame(stats_df.values.reshape(-1)).T.reset_index(drop=True)
    reshaped_df.columns = column_names_list

    return reshaped_df

In [34]:
def uncommon_groupby_feat_engi(df, col_to_group, target_col):
    stats_df = pd.DataFrame(
        df.groupby(col_to_group)[target_col].describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99])
    )

    stats_df['sum'] = df.groupby(col_to_group)[target_col].sum()
    stats_df['entropy'] = df.groupby(col_to_group)[target_col].apply(entropy)

    
    stats_stats_df = stats_df.describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99])
    stats_stats_count = stats_stats_df['count'][0]
    stats_stats_df.drop('count', axis=0, inplace=True)
    stats_stats_df = stats_stats_df.T
    
    column_names_list = []
    for idx in stats_stats_df.index:
        for col in stats_stats_df.columns:
            column_names_list.append(target_col + '_' + col_to_group + '_' + str(idx) + '_' + str(col))
    
    reshaped_df = pd.DataFrame(stats_stats_df.values.reshape(-1)).T.reset_index(drop=True)
    reshaped_df.columns = column_names_list

    current_cols = reshaped_df.columns.tolist()
    count_col_name = target_col + '_' + col_to_group + '_count'
    reshaped_df[count_col_name] = stats_stats_count

    reshaped_df = reshaped_df[[count_col_name] + current_cols]

    return reshaped_df

In [35]:
def read_parquet(dataset='train'):
    parquet_id_folders_list = os.listdir(path + 'series_' + dataset + '.parquet/')
    ts_list = []
    ts_features_list = []
    for id_folder in tqdm(parquet_id_folders_list):
        sibal_id = id_folder.replace('id=', '')
    
        parquet_files_path = path + 'series_' + dataset + '.parquet/' + id_folder + '/'
        parquet_files_list = os.listdir(parquet_files_path)
    
        dfs_list = []
        for parquet_filename in parquet_files_list:
            if '.parquet' in parquet_filename:
                parquet_df = pd.read_parquet(parquet_files_path + parquet_filename)
                dfs_list.append(parquet_df)
        parquet_concat_df = pd.concat(dfs_list, ignore_index=True)
        current_cols = parquet_concat_df.columns.tolist()
        
        parquet_concat_df['id'] = sibal_id
        
        parquet_concat_df = parquet_concat_df[['id'] + current_cols]
        
        parquet_concat_df['day_since_wear'] = (parquet_concat_df['relative_date_PCIAT'] - parquet_concat_df['relative_date_PCIAT'].min()).astype(int)
        parquet_concat_df['time_of_day_sec'] = parquet_concat_df['time_of_day'] / 1e9
        parquet_concat_df.drop('time_of_day', axis=1, inplace=True)
        parquet_concat_df['time_of_day_min'] = parquet_concat_df['time_of_day_sec'] / 60
        parquet_concat_df['time_of_day_hour'] = parquet_concat_df['time_of_day_min'] / 60
        parquet_concat_df['time_of_day_day'] = parquet_concat_df['time_of_day_hour'] / 24
        parquet_concat_df['time_of_day_hour_window'] = parquet_concat_df['time_of_day_hour'].astype(int)
        parquet_concat_df['time_of_day_min_window'] = parquet_concat_df['time_of_day_min'].astype(int)
        parquet_concat_df['time_of_day_15_min_window'] = (parquet_concat_df['time_of_day_min'] / 15).astype(int)
        parquet_concat_df['timestamp_day'] = parquet_concat_df['day_since_wear'] + parquet_concat_df['time_of_day_day']
    
        if parquet_concat_df['timestamp_day'].nunique() != parquet_concat_df.shape[0]:
            print('yes')
    
        parquet_concat_df['timestamp_hour'] = parquet_concat_df['timestamp_day'] * 24
        parquet_concat_df['timestamp_min'] = parquet_concat_df['timestamp_day'] * 24 * 60
        parquet_concat_df['timestamp_sec'] = parquet_concat_df['timestamp_day'] * 24 * 60 * 60
        parquet_concat_df['timestamp_15_min'] = parquet_concat_df['timestamp_day'] * 24 * (60/15)
    
        parquet_concat_df['timestamp_hour_window'] = parquet_concat_df['timestamp_hour'].astype(int)
        parquet_concat_df['timestamp_min_window'] = parquet_concat_df['timestamp_min'].astype(int)
        parquet_concat_df['timestamp_15_min_window'] = parquet_concat_df['timestamp_15_min'].astype(int)
    
        battery_voltage_start = parquet_concat_df['battery_voltage'][0]
        parquet_concat_df['battery_use_since_wear'] = -(parquet_concat_df['battery_voltage'] - battery_voltage_start)
        
    
        worn_df = parquet_concat_df[parquet_concat_df['non-wear_flag'] == 0].reset_index(drop=True)
    
        
        feature_cols = ['X', 'Y', 'Z', 'enmo', 'anglez', 'light', 'battery_use_since_wear']
    
        features_df = feat_engi(worn_df, feature_cols)
    
        groupby_cols = ['weekday', 'time_of_day_hour_window', 'time_of_day_15_min_window']
    
        groupby_df_list = []
        for groupby_col in groupby_cols:
            for feature_col in feature_cols:
                groupby_features_df = groupby_feat_engi(worn_df, groupby_col, feature_col)
                groupby_df_list.append(groupby_features_df)
    
        groupby_concat_df = pd.concat(groupby_df_list, axis=1)
    
        uncommon_groupby_cols = ['day_since_wear', 'timestamp_hour_window', 'timestamp_15_min_window']#, 'timestamp_min_window']
    
        uncommon_groupby_df_list = []
        for uncommon_groupby_col in uncommon_groupby_cols:
            for unc_feature_col in feature_cols:
                uncommon_groupby_df = uncommon_groupby_feat_engi(worn_df, uncommon_groupby_col, unc_feature_col)
                uncommon_groupby_df_list.append(uncommon_groupby_df)
    
        features_concat_df = pd.concat([features_df, groupby_concat_df, uncommon_groupby_df], axis=1)
        features_current_cols = features_concat_df.columns.tolist()
        features_concat_df['id'] = sibal_id
        features_concat_df = features_concat_df[['id'] + features_current_cols]
    
        for q in [1,2,3,4]:
            if q in worn_df['quarter'].tolist():
                features_concat_df['data_collected_in_quarter_' + str(q)] = 1
            else:
                features_concat_df['data_collected_in_quarter_' + str(q)] = 0

        features_concat_df['id'] = features_concat_df['id'].astype(str)

        features_concat_df.to_csv(path + 'series_' + dataset + '.parquet/id=' + sibal_id + '/features.csv', index=False)
        
        ts_list.append(parquet_concat_df)
        ts_features_list.append(features_concat_df)
    return ts_list, ts_features_list

In [36]:
train_ts_list, train_ts_features_list = read_parquet()

  0%|                                                                                | 1/996 [00:07<2:04:45,  7.52s/it]

00115b9f
00115b9f


  0%|▏                                                                               | 2/996 [00:15<2:11:53,  7.96s/it]

001f3379
001f3379


  0%|▏                                                                               | 3/996 [00:33<3:28:44, 12.61s/it]

...

ff18b749
ff18b749


100%|█████████████████████████████████████████████████████████████████████████████▉| 995/996 [3:59:50<00:15, 15.96s/it]

ffcd4dbd
ffcd4dbd


100%|██████████████████████████████████████████████████████████████████████████████| 996/996 [4:00:00<00:00, 14.46s/it]

ffed1dd5
ffed1dd5


In [42]:
def temp_read_files():
    temp_dfs_list = []
    for temp_id_folder in tqdm(train_parquet_id_folders_list):
        temp_files_list = os.listdir(path + 'series_train.parquet/' + temp_id_folder + '/')
        for temp_filename in temp_files_list:
            if '.csv' in temp_filename:
                temp_df = pd.read_csv(path + 'series_train.parquet/' + temp_id_folder + '/' + temp_filename, dtype={'id': str})
                temp_dfs_list.append(temp_df)

    return temp_dfs_list

In [43]:
sibal_temp_dfs_list = temp_read_files()

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [21:57<00:00,  1.32s/it]


In [44]:
len(sibal_temp_dfs_list)

996

In [45]:
temp_concat_df = pd.concat(sibal_temp_dfs_list, ignore_index=True)
print(temp_concat_df.shape)
temp_concat_df.head()

(996, 27676)


id    X_mean     X_std     X_min      X_1%      X_3%      X_5%  \
0  00115b9f -0.316384  0.453665 -1.746094 -0.957459 -0.934769 -0.914427   
1  001f3379  0.047271  0.610522 -1.038711 -0.987389 -0.965308 -0.929665   
2  00f332d1  0.239004  0.528992 -1.952594 -0.937417 -0.885141 -0.828783   
3  01085eb3 -0.363822  0.514492 -2.284304 -0.993339 -0.979900 -0.966016   
4  012cadd8  0.018670  0.595251 -2.143912 -0.977178 -0.942561 -0.915178   

      X_10%     X_15%     X_20%     X_25%     X_30%     X_35%     X_40%  \
0 -0.871721 -0.811414 -0.745521 -0.684180 -0.625026 -0.567314 -0.508016   
1 -0.822527 -0.744080 -0.624830 -0.482846 -0.376047 -0.219380 -0.053169   
2 -0.619099 -0.376297 -0.178431 -0.063253  0.013506  0.065460  0.143172   
3 -0.931370 -0.890299 -0.849929 -0.795286 -0.743438 -0.688371 -0.631032   
4 -0.813863 -0.722694 -0.592835 -0.514300 -0.408334 -0.309016 -0.184722   

      X_45%     X_50%     X_55%     X_60%     X_65%     X_70%     X_75%  \
0 -0.437547 -0.366849 -0.306484 -0.255599 -0.193461 -0.108544 -0.010677   
1 -0.040168  0.052813  0.158746  0.274674  0.387562  0.494733  0.597837   
2  0.218311  0.291499  0.377403  0.451753  0.535908  0.610236  0.685092   
3 -0.571515 -0.503552 -0.419841 -0.334197 -0.249098 -0.145403 -0.024163   
4 -0.056190  0.036376  0.143034  0.247260  0.360007  0.444944  0.539581   

      X_80%     X_85%     X_90%     X_95%     X_97%     X_99%     X_max  \
0  0.046125  0.138923  0.380755  0.540839  0.687074  0.889449  1.507865   
1  0.694278  0.786812  0.862478  0.947284  0.978612  0.996103  1.034351   
2  0.752594  0.816693  0.886523  0.950107  0.974539  0.996652  1.666465   
3  0.140853  0.273915  0.454001  0.632677  0.766140  0.891769  1.000692   
4  0.634786  0.732723  0.825133  0.917491  0.953949  0.993068  3.341210   

     Y_mean     Y_std     Y_min      Y_1%      Y_3%      Y_5%     Y_10%  \
0  0.016009  0.502702 -2.905339 -0.977749 -0.928411 -0.877531 -0.730302   
1 -0.003308  0.454752 -1.522690 -0.897626 -0.823287 -0.765601 -0.611894   
2  0.067278  0.488057 -2.361866 -0.969803 -0.935022 -0.897576 -0.716495   
3 -0.057915  0.431997 -2.276082 -0.989564 -0.930472 -0.851888 -0.673733   
4  0.071660  0.508311 -3.373025 -1.048439 -0.918585 -0.826581 -0.649036   

      Y_15%     Y_20%     Y_25%     Y_30%     Y_35%     Y_40%     Y_45%  \
0 -0.584147 -0.430516 -0.309863 -0.224424 -0.136134 -0.054089 -0.010130   
1 -0.491171 -0.407455 -0.309514 -0.225133 -0.148145 -0.090263 -0.044995   
2 -0.487230 -0.318040 -0.211965 -0.118938 -0.036678  0.012682  0.064731   
3 -0.518658 -0.412982 -0.324336 -0.251998 -0.187119 -0.134176 -0.082795   
4 -0.480292 -0.346330 -0.254007 -0.181510 -0.096670 -0.036464  0.019013   

      Y_50%     Y_55%     Y_60%     Y_65%     Y_70%     Y_75%     Y_80%  \
0  0.024974  0.068776  0.122630  0.205260  0.309391  0.400677  0.495396   
1 -0.032031  0.017075  0.052635  0.125170  0.184959  0.257940  0.360792   
2  0.109878  0.157751  0.212424  0.272188  0.338519  0.400713  0.471606   
3 -0.034986  0.006740  0.053148  0.100025  0.154695  0.219121  0.291143   
4  0.096187  0.183083  0.233549  0.294277  0.357776  0.458349  0.544063   

      Y_85%     Y_90%     Y_95%     Y_97%     Y_99%     Y_max    Z_mean  \
0  0.593706  0.702779  0.812206  0.858177  0.934910  1.666354 -0.167890   
1  0.499812  0.678877  0.834630  0.907865  0.986683  1.946303  0.043286   
2  0.561150  0.691220  0.843300  0.909164  0.969124  1.016429  0.343440   
3  0.380603  0.501571  0.670905  0.757785  0.898741  1.011419 -0.300906   
4  0.634750  0.711058  0.828655  0.895826  0.972946  4.442658 -0.061682   

      Z_std     Z_min      Z_1%      Z_3%      Z_5%     Z_10%     Z_15%  \
0  0.585710 -1.048372 -1.010391 -0.993666 -0.957112 -0.897970 -0.821275   
1  0.623933 -1.018787 -1.009865 -1.008992 -1.007377 -0.944074 -0.788080   
2  0.515729 -1.016758 -0.988789 -0.857476 -0.733482 -0.457626 -0.252333   
3  0.520138 -1.022549 -0.987087 -0.970566 -0.954552 -0.915447 -0.871919   
4  0.578022 -1.003249 -0.9

In [49]:
temp_concat_df['id'].isin(train_parquet_id_list).sum()

996

In [ ]:
def process_file(id_folder, dirname):
    files = os.listdir(os.path.join(path, dirname, id_folder))
    if len(files) > 1:
        df_list = []
        for file in files:
            df = pd.read_parquet(os.path.join(dirname, id_folder, file))
            df_list.append(df)
        df = pd.concat(df_list, ignore_index=True)
    else:
        df = pd.read_parquet(os.path.join(dirname, id_folder, files[0]))
    
    df.drop('step', axis=1, inplace=True)

    df = df[df['non-wear_flag'] == 0].drop('non-wear_flag', axis=1).reset_index(drop=True)

    describe_df = df.describe(percentiles=[0.01] + [i*0.5/10 + 0.05 for i in range(19)] + [0.99])
    count_mean = describe_df.loc['count', :].mean()
    describe_df.drop('count', axis=0, inplace=True)
    
    return np.append(describe_df.values.reshape(-1), count_mean), id_folder.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    
    stats, indexes = zip(*results)
    
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df


class AutoEncoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, encoding_dim*4),
            nn.ReLU(),
            nn.Linear(encoding_dim*4, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, encoding_dim*2),
            nn.ReLU(),
            nn.Linear(encoding_dim*2, encoding_dim*4),
            nn.ReLU(),
            nn.Linear(encoding_dim*4, input_dim),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


def perform_autoencoder(df, encoding_dim=50, epochs=50, batch_size=32):
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    
    data_tensor = torch.FloatTensor(df_scaled)
    
    input_dim = data_tensor.shape[1]
    autoencoder = AutoEncoder(input_dim, encoding_dim)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(autoencoder.parameters())
    
    for epoch in range(epochs):
        for i in range(0, len(data_tensor), batch_size):
            batch = data_tensor[i : i + batch_size]
            optimizer.zero_grad()
            reconstructed = autoencoder(batch)
            loss = criterion(reconstructed, batch)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}]')
                 
    with torch.no_grad():
        encoded_data = autoencoder.encoder(data_tensor).numpy()
        
    df_encoded = pd.DataFrame(encoded_data, columns=[f'Enc_{i + 1}' for i in range(encoded_data.shape[1])])
    
    return df_encoded

def feature_engineering(df):
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    
    return df

In [ ]:
train_ts = load_time_series(path + "series_train.parquet")
test_ts = load_time_series(path + "series_test.parquet")

In [ ]:
concat_ts = pd.concat([train_ts, test_ts], ignore_index=True)
df_concat = concat_ts.drop('id', axis=1)

In [ ]:
for col in df_concat.columns:
    if df_concat[col].std() == 0:
        df_concat.drop(col, axis=1, inplace=True)

In [ ]:
time_series_cols = df_concat.columns.tolist()
df_concat["id"] = concat_ts["id"]
train_ts = df_concat.head(train_ts.shape[0]).reset_index(drop=True)
test_ts = df_concat.tail(test_ts.shape[0]).reset_index(drop=True)
train_merge = pd.merge(train, train_ts, how="left", on='id')
test_merge = pd.merge(test, test_ts, how="left", on='id')

In [ ]:
cat_cols = []
for i in range(data_dict.shape[0]):
    col = data_dict['Field'][i]
    if col == 'id':
        continue
    if (data_dict['Type'][i] == 'str') | ('categori' in data_dict['Type'][i]):
        cat_cols.append(col)

In [ ]:
ordinal_variables = ['FGC-FGC_CU_Zone', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone',
                     'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_Frame_num',
                     'PreInt_EduHx-computerinternet_hoursday']

ordinal_variables += [col for col in cat_cols if (('PCIAT' in col)&('Season' not in col))]
print(len(ordinal_variables))
ordinal_variables

In [ ]:
cat_cols = list(set(cat_cols) - set(ordinal_variables))
print(len(cat_cols))
cat_cols

In [ ]:
train_nonan = train_merge.dropna(subset=['sii'], ignore_index=True)
print(train_nonan.shape)
train_nonan.head()

In [ ]:
train_one_hot = train_nonan.copy()
test_one_hot = test_merge.copy()

In [ ]:
for col in cat_cols:
    if 'PCIAT' not in col:
        train_sr = train_one_hot[col].fillna('Missing').apply(str)        
        test_sr = test_one_hot[col].fillna('Missing').apply(str)
        train_one_hot[col] = train_sr
        test_one_hot[col] = test_sr
        concat_sr = pd.concat([train_sr, test_sr])
        unique_list = sorted(concat_sr.unique().tolist())

        for v in unique_list:
            new_col_name = col + '_' + v
            train_one_hot[new_col_name] = pd.Series(train_one_hot[col] == v).astype(int)
            test_one_hot[new_col_name] = pd.Series(test_one_hot[col] == v).astype(int)
    else:
        train_sr = train_one_hot[col].fillna('Missing').apply(str)
        train_one_hot[col] = train_sr
        unique_list = sorted(train_sr.unique().tolist())

        for v in unique_list:
            new_col_name = col + '_' + v
            train_one_hot[new_col_name] = pd.Series(train_one_hot[col] == v).astype(int)

train_one_hot.drop(cat_cols, axis=1, inplace=True)
test_one_hot.drop([col for col in cat_cols if 'PCIAT' not in col], axis=1, inplace=True)

In [ ]:
print(train_ts.shape)
train_ts.head()

In [ ]:
print(train_one_hot.shape)
train_one_hot.head()

In [ ]:
print(test_one_hot.shape)
test_one_hot.head()

In [ ]:
train_one_hot.drop('id', axis=1, inplace=True)
test_one_hot.drop('id', axis=1, inplace=True)

In [ ]:
train = train_one_hot[test_one_hot.columns.tolist() + ['sii']].copy()
test = test_one_hot.copy()

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

In [ ]:
concat_df = pd.concat([train[test.columns], test], ignore_index=True)
print(concat_df.shape)
concat_df.head()

In [ ]:
imputer = KNNImputer(n_neighbors=5)
imputed_data_concat = imputer.fit_transform(concat_df)

In [ ]:
concat_imputed = pd.DataFrame(imputed_data_concat, columns=concat_df.columns)
print(concat_imputed.shape)
concat_imputed.head()

In [ ]:
concat_imputed.isna().sum().sum()

In [ ]:
concat_imputed = feature_engineering(concat_imputed)
print(concat_imputed.shape)
concat_imputed.head()

In [ ]:
if np.any(np.isinf(concat_imputed)):
    print('inf in data')
    concat_imputed = concat_imputed.replace([np.inf, -np.inf], np.nan)

In [ ]:
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(concat_imputed)

In [ ]:
concat_imputed = pd.DataFrame(imputed_data, columns=concat_imputed.columns)
print(concat_imputed.shape)
concat_imputed.head()

In [ ]:
concat_imputed.isna().sum().sum()

In [ ]:
train_imputed = concat_imputed.head(train.shape[0]).reset_index(drop=True)
test_imputed = concat_imputed.tail(test.shape[0]).reset_index(drop=True)

In [ ]:
train_imputed['sii'] = train['sii']

In [ ]:
train_final = train_imputed.copy()
test_final = test_imputed.copy()

In [ ]:
print(train_final.shape)
print(test_final.shape)

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

# Model Training and Evaluation

- **Model Types**: Various models are used, including:
  - **LightGBM**: A gradient-boosting framework known for its speed and efficiency with large datasets.
  - **XGBoost**: Another powerful gradient-boosting model used for structured data.
  - **CatBoost**: Optimized for categorical features without the need for extensive preprocessing.
  - **Voting Regressor**: An ensemble model that combines the predictions of LightGBM, XGBoost, and CatBoost for better accuracy.
- **Cross-Validation**: Stratified K-Folds cross-validation is employed to split the data into training and validation sets, ensuring balanced class distribution in each fold.
- **Quadratic Weighted Kappa (QWK)**: The performance of the models is evaluated using QWK, which measures the agreement between predicted and actual values, taking into account the ordinal nature of the target variable.
- **Threshold Optimization**: The `minimize` function from `scipy.optimize` is used to fine-tune decision thresholds that map continuous predictions to discrete categories (None, Mild, Moderate, Severe).


In [ ]:
def TrainML(model_class, train_data, test_data):
    X = train_data.drop(['sii'], axis=1)
    y = train_data['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission


# Hyperparameter Tuning

- **LightGBM Parameters**: Hyperparameters such as `learning_rate`, `max_depth`, `num_leaves`, and `feature_fraction` are tuned to improve the performance of the LightGBM model. These parameters control the complexity of the model and its ability to generalize to new data.
- **XGBoost and CatBoost Parameters**: Similar tuning is applied for XGBoost and CatBoost, adjusting parameters such as `n_estimators`, `max_depth`, `learning_rate`, `subsample`, and `regularization` terms (`reg_alpha`, `reg_lambda`). These help in controlling overfitting and ensuring the model's robustness.

In [ ]:
Params7 = {
    'learning_rate': 0.03884249148676395,
    'max_depth': 12,
    'num_leaves': 413,
    'min_data_in_leaf': 14,
    'feature_fraction': 0.7987976913702801,
    'bagging_fraction': 0.7602261703576205,
    'bagging_freq': 2,
    'lambda_l1': 4.735462555910575,
    'lambda_l2': 4.735028557007343e-06
} # CV : 0.4094 | LB : 0.471

In [ ]:
Light = LGBMRegressor(**Params7, random_state=SEED, verbose=-1, n_estimators=200)

In [ ]:
Submission1 = TrainML(Light, train_final, test_final)
Submission1.to_csv('submission.csv', index=False)

Submission1